In [3]:
import numpy as np

def func(x):
    return x*(x>0)

def derivFunc(x):
    return x>0

In [16]:
class RELU:
    
    def __init__(self, indims, nodes=6, method='vanilla'):
    #outdims is equal to the number of nodes in current layer, indims is dependent on previous layer.
        #weights    
        self.W = np.random.randn(indims, nodes) * np.sqrt(2/(indims+nodes))
        self.Mw = np.zeros_like(self.W)
        self.Gw = np.ones_like(self.W)
        self.Vw = np.zeros_like(self.W)
        self.Movw = np.zeros_like(self.W)
        #biases
        self.B = np.random.randn(1, nodes) * np.sqrt(2/(indims+nodes))
        self.Mb = np.zeros_like(self.B)
        self.Gb = np.ones_like(self.B)
        self.Vw = np.zeros_like(self.B)
        self.Movb = np.zeros_like(self.B)

    def learnRateUpdate(self):
        self.LRw = self.LRo / np.sqrt(self.Gw+1e-10)
        self.LRb = self.LRo / np.sqrt(self.Gb+1e-10)
        
    def adagrad(self, LR):
        self.Gw = self.Gw + (self.gradw)**2
        self.Gb = self.Gb + (self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.W -= LRw*self.A.T@self.deriv
        self.B -= LRb*np.sum(self.deriv, axis=0) 
        
    def adagradM(self, LR, mu=.9):
        self.Gw = self.Gw + (self.gradw)**2
        self.Gb = self.Gb + (self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.Mw = mu*self.Mw - LRw*self.gradw
        self.Mb = mu*self.Mb - LRb*self.gradb  
        self.W += self.Mw
        self.B += self.Mb 
     
    def adagradNAG(self, LR):
        self.Gw = self.Gw + (self.gradw)**2
        self.Gb = self.Gb + (self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.W -= LRw*self.A.T@self.deriv
        self.B -= LRb*np.sum(self.deriv, axis=0)
        
    def rmsProp(self, LR, gamma=.9):
        self.Gw = gamma*self.Gw + (1-gamma)*(self.gradw)**2
        self.Gb = gamma*self.Gb + (1-gamma)*(self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.W -= LRw*self.A.T@self.deriv
        self.B -= LRb*np.sum(self.deriv, axis=0) 
                 
    def rmsPropM(self, LR, mu=.9, gamma=.9):
        self.Gw = gamma*self.Gw + (1-gamma)*(self.gradw)**2
        self.Gb = gamma*self.Gb + (1-gamma)*(self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.Mw = mu*self.Mw - LRw*self.gradw
        self.Mb = mu*self.Mb - LRb*self.gradb  
        self.W += self.Mw
        self.B += self.Mb 
        
    def rmsPropNAG(self, LR, mu=.9, gamma=.9):
        self.Gw = gamma*self.Gw + (1-gamma)*(self.gradw)**2
        self.Gb = gamma*self.Gb + (1-gamma)*(self.gradb)**2
        LRw = LR / np.sqrt(self.Gw+1e-10)
        LRb = LR / np.sqrt(self.Gb+1e-10)
        self.Mw = mu*self.Mw - LRw*self.gradw
        self.Mb = mu*self.Mb - LRb*self.gradb  
        self.W += self.Mw
        self.B += self.Mb
        
    def adam(self, LR, mu=.9, gamma=.9, momentum=False):
        self.Movw = mu*self.Movw + (1-mu)*(self.gradw)
        self.Movb = mu*self.Movb + (1-mu)*(self.gradb)
        self.Vw = gamma*self.Vw + (1-gamma)*(self.gradw)**2
        self.Vb = gamma*self.Vb + (1-gamma)*(self.gradb)**2
        LRw = LR / np.sqrt(self.Vw+1e-10)
        LRb = LR / np.sqrt(self.Vb+1e-10)
        self.W -= LRw*self.A.T@self.deriv
        self.B -= LRb*np.sum(self.deriv, axis=0)        
    
    def momentum(self,LR, mu):    
        self.Mw = mu*self.Mw - LR*self.gradw
        self.Mb = mu*self.Mb - LR*self.gradb  
        self.W += self.Mw
        self.B += self.Mb
        
    def momentumDecay(self,LR, mu):    
        self.Mw = mu*self.Mw - LR*self.gradw
        self.Mb = mu*self.Mb - LR*self.gradb  
        self.W += self.Mw
        self.B += self.Mb
        
    def vanillaWU(self, LR):
        self.W -= LR*self.A.T@self.deriv
        self.B -= LR*np.sum(self.deriv, axis=0)        
                
#####propagation methods#####

    def forward(self, X):
        self.A = X
        self.H = self.A@self.W+self.B
        self.Z = func(self.H)
        return self.Z
        #store As, Hs, and Zs
            
    def backward(self, Dn, LR):
        #set derivative
        self.LR = LR
        deriv = Dn*derivFunc(self.H)
        self.gradw = self.A.T@deriv
        self.gradb = np.sum(deriv,axis=0)
        D = deriv*self.W.T        
        return D
        #optimizations go here
            
    def NAGforward(self, X):
        self.A = X
        self.H = self.A@self.W+self.B
        self.Z = func(self.H)
        return self.Z
    
    def NAGbackward(self, Dn):
        #set derivative
        deriv = Dn*derivFunc(self.H)
        self.gradw = self.A.T@deriv
        self.gradb = np.sum(deriv,axis=0)
        D = deriv*self.W.T
        return D
        


10
